In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/Colab Notebooks/kaggle/playground-series-s3e2/sample_submission.csv
/content/drive/MyDrive/Colab Notebooks/kaggle/playground-series-s3e2/train.csv
/content/drive/MyDrive/Colab Notebooks/kaggle/playground-series-s3e2/test.csv


#*data exploration*

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/playground-series-s3e2/train.csv')

train_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [ ]:
train_df.groupby('stroke').count()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
stroke,,,,,,,,,,,
0,14672,14672,14672,14672,14672,14672,14672,14672,14672,14672,14672
1,632,632,632,632,632,632,632,632,632,632,632


*balancing data*

In [ ]:
# Technique 2 Up-sample minority class
from sklearn.utils import resample

print(train_df['stroke'].value_counts())

#Separate majority and minority classes

df_majority = train_df[train_df['stroke'] == 0]
df_minority = train_df[train_df['stroke'] == 1]

# Upsample minority class and other classes separately
# If not, random samples from combined classes will be duplicated and we run into
#same issue as before, undersampled remians undersampled.
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=14672,    # to match average class
                                 random_state=42) # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
print(df_upsampled['stroke'].value_counts())
# df_upsampled

0    14672
1      632
Name: stroke, dtype: int64
0    14672
1    14672
Name: stroke, dtype: int64


In [ ]:
train_df=df_upsampled

In [ ]:
y_train = df_upsampled['stroke']

#convert data from categorical to numerical 

In [ ]:
train_df['gender'].unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [ ]:
train_df['ever_married'].unique()

array(['Yes', 'No'], dtype=object)

In [ ]:
train_df['work_type'].unique()

array(['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked'],
      dtype=object)

In [ ]:
train_df['Residence_type'].unique()

array(['Urban', 'Rural'], dtype=object)

In [ ]:
train_df['smoking_status'].unique()

array(['never smoked', 'formerly smoked', 'Unknown', 'smokes'],
      dtype=object)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/playground-series-s3e2/test.csv')
test_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,15304,Female,57.0,0,0,Yes,Private,Rural,82.54,33.4,Unknown
1,15305,Male,70.0,1,0,Yes,Private,Urban,72.06,28.5,Unknown
2,15306,Female,5.0,0,0,No,children,Urban,103.72,19.5,Unknown
3,15307,Female,56.0,0,0,Yes,Govt_job,Urban,69.24,41.4,smokes
4,15308,Male,32.0,0,0,Yes,Private,Rural,111.15,30.1,smokes


In [ ]:
test_df_id = test_df['id']

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29344 entries, 0 to 1274
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 29344 non-null  int64  
 1   gender             29344 non-null  object 
 2   age                29344 non-null  float64
 3   hypertension       29344 non-null  int64  
 4   heart_disease      29344 non-null  int64  
 5   ever_married       29344 non-null  object 
 6   work_type          29344 non-null  object 
 7   Residence_type     29344 non-null  object 
 8   avg_glucose_level  29344 non-null  float64
 9   bmi                29344 non-null  float64
 10  smoking_status     29344 non-null  object 
 11  stroke             29344 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 2.9+ MB


In [ ]:
cat_df_1 = train_df[['gender']]
cat_test_df_1 = test_df[['gender']]
cat_df_2 = train_df[['work_type']]
cat_test_df_2 = test_df[['work_type']]
cat_df_3 = train_df[['Residence_type']]
cat_test_df_3 = test_df[['Residence_type']]
cat_df_4 = train_df[['smoking_status']]
cat_test_df_4 = test_df[['smoking_status']]
cat_df_5 = train_df[['ever_married']]
cat_test_df_5 = test_df[['ever_married']]
cat_df_1.head()

,gender
0,Male
1,Male
2,Female
3,Male
4,Female


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Create an instance of the encoder
encoder1 = OneHotEncoder()
encoder2 = OneHotEncoder()
encoder3 = OneHotEncoder()
encoder4 = OneHotEncoder()
encoder5 = OneHotEncoder()
dumm_df_1 = pd.DataFrame(encoder1.fit_transform(cat_df_1).toarray(),columns = encoder1.get_feature_names_out(cat_df_1.columns))
dumm_test_df_1= pd.DataFrame(encoder1.transform(cat_test_df_1).toarray(),columns = encoder1.get_feature_names_out(cat_test_df_1.columns))
dumm_df_2 = pd.DataFrame(encoder2.fit_transform(cat_df_2).toarray(),columns = encoder2.get_feature_names_out(cat_df_2.columns))
dumm_test_df_2= pd.DataFrame(encoder2.transform(cat_test_df_2).toarray(),columns = encoder2.get_feature_names_out(cat_test_df_2.columns))
dumm_df_3 = pd.DataFrame(encoder3.fit_transform(cat_df_3).toarray(),columns = encoder3.get_feature_names_out(cat_df_3.columns))
dumm_test_df_3= pd.DataFrame(encoder3.transform(cat_test_df_3).toarray(),columns = encoder3.get_feature_names_out(cat_test_df_3.columns))
dumm_df_4 = pd.DataFrame(encoder4.fit_transform(cat_df_4).toarray(),columns = encoder4.get_feature_names_out(cat_df_4.columns))
dumm_test_df_4= pd.DataFrame(encoder4.transform(cat_test_df_4).toarray(),columns = encoder4.get_feature_names_out(cat_test_df_4.columns))
dumm_df_5 = pd.DataFrame(encoder5.fit_transform(cat_df_5).toarray(),columns = encoder5.get_feature_names_out(cat_df_5.columns))
dumm_test_df_5= pd.DataFrame(encoder5.transform(cat_test_df_5).toarray(),columns = encoder5.get_feature_names_out(cat_test_df_5.columns))

concat_dumm = pd.concat([dumm_df_2,dumm_df_3,dumm_df_4],axis = 1)
concat_dumm_test = pd.concat([dumm_test_df_2,dumm_test_df_3,dumm_test_df_4],axis = 1)
concat_dumm

,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
29339,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
29340,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
29341,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
29342,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
num_df_1 = train_df[['age']]
num_test_df_1 = test_df[['age']]
num_df_2 = train_df[['avg_glucose_level']]
num_test_df_2 = test_df[['avg_glucose_level']]
num_df_3 = train_df[['bmi']]
num_test_df_3 = test_df[['bmi']]
num_df_1.head()

,age
0,28.0
1,33.0
2,42.0
3,56.0
4,24.0


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 =  StandardScaler()

scaler3 =  StandardScaler()
scaled_df_1 = pd.DataFrame(scaler1.fit_transform(num_df_1),columns = num_df_1.columns)
scaled_test_df_1 =  pd.DataFrame(scaler1.transform(num_test_df_1),columns = num_test_df_1.columns)
scaled_df_2 =  pd.DataFrame(scaler2.fit_transform(num_df_2),columns = num_df_2.columns)
scaled_test_df_2 =  pd.DataFrame(scaler2.transform(num_test_df_2) ,columns = num_test_df_2.columns)
scaled_df_3 =  pd.DataFrame(scaler3.fit_transform(num_df_3),columns = num_df_3.columns)
scaled_test_df_3 =  pd.DataFrame(scaler3.transform(num_test_df_3),columns = num_test_df_3.columns)

concat_scaled_df = pd.concat([scaled_df_1,scaled_df_2,scaled_df_3],axis = 1)
concat_test_scaled_df = pd.concat([scaled_test_df_1,scaled_test_df_2,scaled_test_df_3],axis =1)
concat_test_scaled_df

,age,avg_glucose_level,bmi
0,0.124527,-0.371233,0.684770
1,0.713437,-0.626922,-0.093690
2,-2.231115,0.145514,-1.523516
3,0.079226,-0.695724,1.955726
4,-1.007993,0.326790,0.160501
...,...,...,...
10199,-1.234497,-0.536406,-1.825368
10200,-0.237880,0.125751,-0.379655
10201,-2.321717,0.153321,-1.714159
10202,-1.053294,-0.374404,-0.061916


In [ ]:
rem_two_cols  = train_df[['hypertension','heart_disease']]
rem_two_cols_test = test_df[['hypertension','heart_disease']]
rem_two_cols_test.head()

,hypertension,heart_disease
0,0,0
1,1,0
2,0,0
3,0,0
4,0,0


In [ ]:
concat_dumm.shape,concat_scaled_df.shape,rem_two_cols.shape

((29344, 11), (29344, 3), (29344, 2))

In [ ]:
rem_two_cols

,hypertension,heart_disease
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
5466,0,0
8967,1,0
6868,0,0
8000,0,0


In [ ]:
X_train = pd.concat([concat_dumm.reset_index(drop=True),concat_scaled_df.reset_index(drop=True),rem_two_cols.reset_index(drop=True)],axis =1)
#X_train = pd.concat([X_train,rem_two_cols],axis=1)

In [ ]:
X_test = pd.concat([concat_dumm_test.reset_index(drop=True),concat_test_scaled_df.reset_index(drop=True),rem_two_cols_test.reset_index(drop=True)],axis =1)
X_test

,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,age,avg_glucose_level,bmi,hypertension,heart_disease
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.124527,-0.371233,0.684770,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.713437,-0.626922,-0.093690,1,0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,-2.231115,0.145514,-1.523516,0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.079226,-0.695724,1.955726,0,0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.007993,0.326790,0.160501,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10199,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.234497,-0.536406,-1.825368,0,0
10200,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.237880,0.125751,-0.379655,0,0
10201,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-2.321717,0.153321,-1.714159,0,0
10202,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.053294,-0.374404,-0.061916,0,0


#model building(XGBoost) 

In [ ]:
from sklearn.model_selection import  train_test_split
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42,stratify=y_train)

In [ ]:
import time
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

searching best parameters 

In [ ]:
import time
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
clf=xgb.XGBClassifier()

param_grid = {
        'learning_rate': [ 0.01,0.03, 0.1,0.3, 0.5,0.8, 1,2,5,10],
        'gamma': [0.1,0.3, 0, 0.25, 0.5, 1.0,2.0,3.0,5.0,7.0],
        'reg_lambda': [0.1,0.3, 1.0,1.3, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [3,5,10,20,30,40,50,70,100]}


rs_clf = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=30)
rs_clf.fit(X_train, y_train)
 
# Identify optimal hyperparameter values
best_n_estim= rs_clf.best_params_['n_estimators']
print("The best parameters: {}".format(rs_clf.best_params_))

The best parameters: {'reg_lambda': 5.0, 'n_estimators': 50, 'learning_rate': 1, 'gamma': 0.5}


In [ ]:
xgb_classifier = xgb.XGBClassifier(reg_lambda= 1.0, n_estimators= 50, learning_rate= 1, gamma= 0)
#xgb_classifier = xgb.XGBClassifier(reg_lambda= 5.0, n_estimators= 50, learning_rate= 1, gamma= 0.5)
xgb_classifier.fit(X_train, y_train)
xgb_predictions = xgb_classifier.predict(X_val)
 
print (classification_report(y_val, xgb_predictions))

              precision    recall  f1-score   support

           0       0.96      0.86      0.91      2935
           1       0.87      0.97      0.92      2934

    accuracy                           0.91      5869
   macro avg       0.92      0.91      0.91      5869
weighted avg       0.92      0.91      0.91      5869



In [ ]:
print(xgb_classifier.score())

In [ ]:
# from joblib import dump, load
# path='/content/drive/MyDrive/Colab Notebooks/kaggle/playground-series-s3e2'
# dump(xgb_classifier, os.path.join(path,'filename.joblib')) 

['/content/drive/MyDrive/Colab Notebooks/kaggle/playground-series-s3e2/filename.joblib']

In [ ]:
# xgb_classifier = load(os.path.join(path,'filename.joblib')) 
# xgb_classifier.fit(X_train, y_train)
# xgb_predictions = xgb_classifier.predict(X_val)
 
# print (classification_report(y_val, xgb_predictions))

              precision    recall  f1-score   support

           0       0.96      0.86      0.91      2935
           1       0.87      0.97      0.92      2934

    accuracy                           0.91      5869
   macro avg       0.92      0.91      0.91      5869
weighted avg       0.92      0.91      0.91      5869



In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

classifier.fit(X_train,y_train)

LogisticRegression()

In [ ]:
classifier.score(X_train,y_train)

0.8036634717784877